In [207]:
import os
import sys
# Locate the script in UnifiedInflation/examples and add to path one folder before, that is, UnifiedInflation/
# in order to be able to import quantuminflation
# ! Note: I found online that "__file__" sometimes can be problematic, So I'm using the solution provided in
# https://stackoverflow.com/questions/2632199/how-do-i-get-the-path-of-the-current-executed-file-in-python?lq=1
from inspect import getsourcefile
from os.path import abspath
from tabnanny import verbose

#cws = abspath(getsourcefile(lambda:0))
cws = os.getcwd()
cws = os.sep.join(cws.split(os.sep)[:-1])  # Remove the script filename to get the directory of the script
cws = cws + os.sep + os.pardir  # Go one folder above UnifiedInflation/examples -> UnifiedInflation/
sys.path.append(cws)
###############################################################################

import itertools
import pickle

import numpy as np
from quantuminflation.InflationProblem import InflationProblem
from quantuminflation.InflationSDP import InflationSDP
from quantuminflation.useful_distributions import *
from quantuminflation.general_tools import to_numbers

from quantuminflation.fast_npa import calculate_momentmatrix
import itertools

from ncpol2sdpa.nc_utils import apply_substitutions
import sympy as sp

# Testing

Check if $U_{0|0}$, $U_{0|1}$ and $V$ are valid NBF over AQ. We'll try to find the maximum and minimum over AQ.

First $U_{0|0}$, $U_{0|1}$

!! After checking the first time, we renormalize to machine precision.

Now lets check $V$

## Check CHSH with the SDP

In [213]:
InfProb = InflationProblem(hypergraph=[[1, 1]], outcomes_per_party=[2,2], settings_per_party=[2,2], inflation_level_per_source=[1],names=['A', 'B'])
InfSDP = InflationSDP(InfProb, commuting=False, verbose=0)
InfSDP.generate_relaxation( column_specification='local1',
                            max_monomial_length=False,
                            find_physical_monomials=False,
                            use_linear_constraints=False,
                            sandwich_positivity=False,
                            load_from_file=False,
                            filename_label=filename_label)
meas = InfSDP.measurements
A0, A1, B0, B1 = 1-2*meas[0][0][0][0], 1-2*meas[0][0][1][0], 1-2*meas[1][0][0][0], 1-2*meas[1][0][1][0]
obj =  A0*B0+A0*B1+A1*B0-A1*B1
obj = sp.expand(obj)
print(obj)
InfSDP.set_objective(objective=+obj)
InfSDP.solve(interpreter='CVXPY')  # By default it maximizes
print("minimum over AQ of CHSH", InfSDP.primal_objective)

Reassigning moment matrix indices: 100%|██████████| 9/9 [00:00<?, ?it/s]

2 - 4*A_1_0_0 + 4*A_1_0_0*B_1_0_0 + 4*A_1_0_0*B_1_1_0 + 4*A_1_1_0*B_1_0_0 - 4*A_1_1_0*B_1_1_0 - 4*B_1_0_0
minimum over AQ of CHSH 2.8284271790023405
